In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [5]:
import pickle,os
import trako as TKO

In [1]:
SUBJECT='/'
BLACKLIST = 'tracts_commissural'
DATAFOLDER = '/home/haehn/Dropbox/TRAKODATA/AnatomicalTracts_VTK//'
# DATAFOLDER = '/home/haehn/Dropbox/TKOTEST/'
DATAFOLDER_TKO = DATAFOLDER[:-1]+'_TKO/'
DATAFOLDER_RESTORED = DATAFOLDER[:-1]+'_RESTORED/'
STATSFOLDER = DATAFOLDER[:-1]+'_STATS/'
DATAFOLDER += SUBJECT
DATAFOLDER_TKO += SUBJECT
DATAFOLDER_RESTORED += SUBJECT
STATSFILE = STATSFOLDER+SUBJECT[:-1]+'.p'

originalsize = []
compressedsize = []
number_of_streamlines = []
c_time = []
d_time = []


In [22]:
THRESHOLD = 10
scalarstats = {
    
}
propertystats = {
    
}
counter = 0
firstrun = True
t0 = time.time()
for f in os.listdir(DATAFOLDER):

        if name.endswith('.vtk'):
            input = os.path.join(root, name)
            compressed = input.replace('.vtk','.tko')
            restored = input.replace('')
            
            i_poly = TKO.Util.loadvtp(input)
            i_scalars = i_poly['scalars']
            i_nscalars = i_poly['scalar_names']
            i_properties = i_poly['properties']
            i_nproperties = i_poly['property_names']
            
            r_poly = TKO.Util.loadvtp(restored)
            r_scalars = r_poly['scalars']
            r_properties = r_poly['properties']
            
#             print(i_poly['points'])
#             print(r_poly['points'])
#             print(i_scalars)
#             print(r_scalars)

            if len(i_scalars)==0 or len(r_scalars) == 0:
                print('skipped', name)
                continue
                
#             if (np.isnan(np.any(i_scalars))):
#                 print('NaN', name, i_scalars)
#                 continue
                
#             if (np.isnan(np.any(r_scalars))):
#                 print('NaN2', name, i_scalars)
#                 continue
            
            if firstrun:
                for i,s in enumerate(i_nscalars):
                    scalarstats[s] = [0,0,0,0,np.inf,-np.inf]
#                     print ('reset')
                for i,p in enumerate(i_nproperties):
                    propertystats[p] = [0,0,0,0,np.inf,-np.inf]
                
                firstrun = False
    
    
            
            for i,s in enumerate(i_scalars):
                c_name = i_nscalars[i]
            
                stats = TKO.Util.error(i_scalars[i], r_scalars[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

                scalarstats[c_name][0] += stats[0][0]
                scalarstats[c_name][1] += stats[0][1]
                scalarstats[c_name][2] += stats[0][2]
                scalarstats[c_name][3] += stats[0][3]
                scalarstats[c_name][4] = min(np.min(i_scalars[i]),scalarstats[c_name][4] )
                scalarstats[c_name][5] = max(np.max(i_scalars[i]),scalarstats[c_name][5] )

            for i,s in enumerate(i_properties):
                c_name = i_nproperties[i]
            
                stats = TKO.Util.error(i_properties[i], r_properties[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

                propertystats[c_name][0] += stats[0][0]
                propertystats[c_name][1] += stats[0][1]
                propertystats[c_name][2] += stats[0][2]
                propertystats[c_name][3] += stats[0][3]
                propertystats[c_name][4] = min(np.min(i_properties[i]),propertystats[c_name][4] )
                propertystats[c_name][5] = max(np.max(i_properties[i]),propertystats[c_name][5] )
                
            counter += 1
        if counter > THRESHOLD:
            break
            
# normalize
for s in scalarstats.keys():
    for i,v in enumerate(scalarstats[s]):
        if i > 3:
            continue
        scalarstats[s][i] /= counter
for s in propertystats.keys():
    for i,v in enumerate(propertystats[s]):
        if i > 3:
            continue
        propertystats[s][i] /= counter
print(time.time()-t0)

106.44178652763367


In [21]:
scalarstats

{'NormalizedSignalEstimationError': [0.0,
  1.5172938054258174e-06,
  7.513314595929511e-07,
  4.3964107279944903e-07,
  0.0,
  0.004544852470809763],
 'EstimatedUncertainty': [0.0,
  0.6063787286931818,
  0.2999116670001637,
  0.1764529672535983,
  0.0035208941183306956,
  2821.9683948863635],
 'RTOP1': [0.0,
  0.0783505223014138,
  0.03998693310528655,
  0.022720706442752005,
  0.10258631272749467,
  2172.903409090909],
 'RTOP2': [0.0,
  0.028653058138760654,
  0.014421229388310829,
  0.008322321659017523,
  0.12034710970791904,
  786.4951171875],
 'RTAP1': [0.0,
  0.06340096213600853,
  0.03144508278769949,
  0.01804910872306209,
  -1231.0636541193182,
  719.5413707386364],
 'RTAP2': [0.0,
  0.020645011555064808,
  0.010295246726441705,
  0.005988844653157602,
  0.10087298263203014,
  620.0487837357955],
 'RTPP1': [0.0,
  0.00013278289274735883,
  6.654764415130062e-05,
  3.83428574754031e-05,
  0.06462655826048418,
  0.8978890505704012],
 'RTPP2': [0.0,
  0.00020793351260098544,
  

In [20]:
propertystats

{'EmbeddingCoordinate': [0.00015848922471791,
  0.0003186018888796256,
  0.00024061065579933876,
  3.267314109094018e-05,
  -0.28866897929798474,
  0.4480217153375799],
 'ClusterNumber': [0.0, 0.0, 0.0, 0.0, 1.0909090909090908, 69.81818181818181],
 'EmbeddingColor': [0.0,
  1.4142135623730956,
  0.8690054731679054,
  0.5114847917885469,
  0.18181818181818182,
  16.363636363636363],
 'TotalFiberSimilarity': [0.0,
  11.502840909090908,
  5.598667795007879,
  3.3409415700218896,
  13625.14346590909,
  63300.57386363636],
 'MeasuredFiberSimilarity': [0.0,
  1.574781807986173e-08,
  7.784084568837361e-09,
  4.740402414039512e-09,
  0.00032631117342547935,
  0.0004941430789503185]}

In [53]:
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],3),np.round(scalarstats[p][5],3)
    mean= np.round(scalarstats[p][2],3),np.round(scalarstats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],3),np.round(scalarstats[p][5],3)
    mean= np.round(scalarstats[p][2],3),np.round(scalarstats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

NormalizedSignalEstimationError ($N$, range:  0.0-0.05 ) \\
EstimatedUncertainty ($N$, range:  0.04-31041.65 ) \\
RTOP1 ($N$, range:  1.13-23901.94 ) \\
RTOP2 ($N$, range:  1.32-8651.45 ) \\
RTAP1 ($N$, range:  -13541.7-7914.96 ) \\
RTAP2 ($N$, range:  1.11-6820.54 ) \\
RTPP1 ($N$, range:  0.71-9.88 ) \\
RTPP2 ($N$, range:  0.71-15.96 ) \\
SignalMean ($N$, range:  0.0-0.04 ) \\
0.0$\pm$0.0 \\
0.3$\pm$0.176 \\
0.04$\pm$0.023 \\
0.014$\pm$0.008 \\
0.031$\pm$0.018 \\
0.01$\pm$0.006 \\
0.0$\pm$0.0 \\
0.0$\pm$0.0 \\
0.0$\pm$0.0 \\


In [54]:
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],3),np.round(propertystats[p][5],3)
    mean= np.round(propertystats[p][2],3),np.round(propertystats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],3),np.round(propertystats[p][5],3)
    mean= np.round(propertystats[p][2],3),np.round(propertystats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

EmbeddingCoordinate ($N$, range:  -3.18-4.93 ) \\
ClusterNumber ($N$, range:  12-768 ) \\
EmbeddingColor ($N$, range:  2-180 ) \\
TotalFiberSimilarity ($N$, range:  149876.58-696306.3 ) \\
MeasuredFiberSimilarity ($N$, range:  0.0-0.0 ) \\
0.0$\pm$0.0 \\
0.0$\pm$0.0 \\
0.869$\pm$0.511 \\
5.599$\pm$3.341 \\
0.0$\pm$0.0 \\


In [23]:
compressedsize = 50462344
originalsize = 149677524

c_ratio = (1-float(compressedsize)/float(originalsize))*100
c_factor = float(originalsize) / float(compressedsize)

In [30]:
np.round(c_ratio,3)

66.286

In [31]:
np.round(c_factor,3)

2.966

In [29]:
np.round(compressedsize/1000,2)

50462.34